# Cluster Size Distribution Estimates

## Imports

In [2]:
!pip install crtoolbox
!pip install numpy
!pip install matplotlib


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# Basic imports
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

# Import supporting functions
from crtoolbox.lib.boundary import *
from crtoolbox.lib.regression import *

# Import bootstrap functions
from crtoolbox.bootstrap import *

# Import data generation
from crtoolbox.tests.generate_2d_data import *

## Data Generation

Below we generate our signal, $\mu$.

In [ ]:
# This command will define a circular smooth signal
signal = CircleSignal(r=25, fwhm=[10,10], mag=3)

# Generate signal
signal.generate()

# Get signal
mu = signal.mu

# Threshold c
c = 2

# Once we have defined a signal we can plot the output like so:
signal.plot()

Below we generate our noise, $\epsilon$.

In [ ]:
# Specify the noise
noise = Noise(var=1,fwhm=[6,6],n=100)

# Plot a single realisation of the noise
noise.plot()

We now generate some data instances.

In [ ]:
# Output directory (feel free to change this to your desired output directory)
out_dir = os.path.join(os.getcwd(),'data','example_2D')

# We can generate some 2D data using the generate_data_2D function
data_files, mu_file = generate_data_2D(signal, noise, out_dir=out_dir)

# Let's record the number of observations; this will be useful later
n = len(data_files)

## Empirical Distribution (Single Instance)

We now need to get the observed $|G|$.

In [ ]:
# Construct a design matrix with only an intercept (this means that the parameter estimate from the regression will be the mean of the data)
X = np.ones((n,1))

# Perform regression to get residuals, muhat, and sigma
muhat_file, sigma_file, resid_files = regression(data_files, X, out_dir=out_dir)

# Read in muhat and sigma
muhat = read_image(muhat_file)
sigma = read_image(sigma_file)

# Compute G = (muhat-mu)/(sigma/sqrt(n))
G = (muhat-mu)/(sigma/np.sqrt(n))

# Make image of G
plt.imshow(G)
plt.colorbar()

We need to now estimate $\tau_n^{-1}(|\hat{\mathcal{K}}^1_c|-|\mathcal{K}^1_c|)$.

In [ ]:
# Get tau
tau = 1/np.sqrt(n) 

# Ac 
Ac = mu > c

# AcHat
AcHat = muhat > c

# Get size of estimated cluster
est_clus_size = np.sum(AcHat)

# Get size of true cluster
true_clus_size = np.sum(Ac)

# Get estimated clt value
clt_est = (est_clus_size - true_clus_size)/tau

We also need to do this using the thickened sets (as the above will always underestimate the true sizes).

In [ ]:
def thicken_set(binary_map):

    # Generate boundary map
    bdry = get_bdry_map_combined(binary_map)
    
    # Add bdry to binary map and rethreshold
    binary_map = np.logical_or(binary_map, bdry)

    # Return binary map
    return binary_map

# Thicken sets
Ac_thickened = thicken_set(Ac)
AcHat_thickened = thicken_set(AcHat)

# Get size of estimated cluster
est_clus_size_thickened = np.sum(AcHat_thickened)

# Get size of true cluster
true_clus_size_thickened = np.sum(Ac_thickened)

# Get estimated clt value
clt_est_thickened = (est_clus_size_thickened - true_clus_size_thickened)/tau

We now get the boundary locations.

In [ ]:
# Get boolean maps for the boundary of AcHat
AcHat_bdry_map = get_bdry_maps(muhat, c)

# Get coordinates for the boundary of AcHat
AcHat_bdry_locs = get_bdry_locs(AcHat_bdry_map)

# Obtain the muhat values along the boundary for AcHat
muhat_bdry_vals_concat = get_bdry_values_concat(muhat, AcHat_bdry_locs)

# Obtain the weights along the boundary for AcHat
AcHat_bdry_weights_concat = get_bdry_weights_concat(muhat_bdry_vals_concat, c)

# Sanity check
plt.imshow(AcHat_bdry_map[0]['bottom']['outer'] + AcHat_bdry_map[0]['bottom']['inner']
        + AcHat_bdry_map[0]['top']['outer'] + AcHat_bdry_map[0]['top']['inner']
        + AcHat_bdry_map[1]['bottom']['outer'] + AcHat_bdry_map[1]['bottom']['inner']
        + AcHat_bdry_map[1]['top']['outer'] + AcHat_bdry_map[1]['top']['inner'])

Now we get $G$ along this boundary.

In [ ]:
# Get G values along the boundary of AcHat
G_bdry_vals_concat = get_bdry_values_concat(G, AcHat_bdry_locs)

# Interpolate G along the boundary of AcHat
G_bdry_interp_concat = get_bdry_vals_interpolated_concat(G_bdry_vals_concat, AcHat_bdry_weights_concat)

# Plot a histogram of the interpolated values
_ = plt.hist(G_bdry_interp_concat, bins=100)

Interpolate sigma along the boundary.

In [ ]:
# Obtain the sigmahat values along the boundary for AcHat
sigma_bdry_vals_concat = get_bdry_values_concat(sigma, AcHat_bdry_locs)

# Interpolate sigma along the boundary of AcHat
sigma_bdry_interp_concat = get_bdry_vals_interpolated_concat(sigma_bdry_vals_concat, AcHat_bdry_weights_concat)

Estimate the gradient of $\mu$ along the boundary by gettin the interpolation weights.

In [ ]:
# Obtain the gradient of muhat from numpy
muhat_grad = np.gradient(muhat)

# Obtain the magnitude of the gradient from the partial derivatives
muhat_grad = np.sqrt(muhat_grad[0]**2 + muhat_grad[1]**2)

# Get muhat_grad values along the boundary of AcHat
muhat_grad_bdry_vals_concat = get_bdry_values_concat(muhat_grad, AcHat_bdry_locs)

# Interpolate muhat_grad along the boundary of AcHat
muhat_grad_bdry_interp_concat = get_bdry_vals_interpolated_concat(muhat_grad_bdry_vals_concat, AcHat_bdry_weights_concat)

Estimate the integral

$$\int_{s \in \partial\mathcal{K}^i_{c}}  \sigma(s)\frac{G(s)}{|\nabla \mu(s)|} d H_{n-1}(s)$$

In [ ]:
integral_value = np.sum(sigma_bdry_interp_concat*G_bdry_interp_concat/muhat_grad_bdry_interp_concat)

print(integral_value)
print(clt_est)

## Empirical Distribution (Bootstrap)

Get interpolated residuals.

In [ ]:
# Loop through subjects
for j in np.arange(n):

    # Obtain residuals
    resid = read_images(resid_files[j])[...,0]

    # Standardize residuals
    resid = (resid/sigma).reshape((1,) + resid.shape)

    # Residuals along boundary for current subject
    resid_bdry_concat_j = get_bdry_values_concat(resid, AcHat_bdry_locs)

    # Concatenate residuals
    if j == 0:
        resids = resid_bdry_concat_j
    else:
        resids = np.concatenate((resids, resid_bdry_concat_j), axis=0)

# Dimensions of bootstrap variables
boot_dim = np.array([n, 1, 1]) 

# Number of bootstraps
n_boot = 5000

We are now ready to perform the bootstrap.

In [ ]:
# Initialize array for integral values
integral_values = []

# Perform bootstrap
for b in np.arange(n_boot):

    # Obtain bootstrap variables
    boot_vars = 2*np.random.randint(0,2,boot_dim,dtype="int8")-1

    # ------------------------------------------------------
    # Bootstrap residuals
    # ------------------------------------------------------

    # Multiply by rademacher variables
    boot_resids = boot_vars*resids

    # ------------------------------------------------------
    # Get gi along dalpha FcHat
    # ------------------------------------------------------
    # Sum across subjects to get the bootstrapped a values along
    # the boundary of dalphaFcHat. (Note: For some reason this is 
    # much faster if performed seperately for each of the last rows. 
    # I am still looking into why this is)
    mean_outer = np.sum(boot_resids[...,0], axis=0)/n
    mean_inner = np.sum(boot_resids[...,1], axis=0)/n

    # Get sum of squares
    ssq_outer = np.sum((boot_resids[...,0]-mean_outer)**2, axis=0)/(n-1)
    ssq_inner = np.sum((boot_resids[...,1]-mean_inner)**2, axis=0)/(n-1)

    # Obtain bootstrap G
    boot_G_outer = np.sqrt(n)*mean_outer/np.sqrt(ssq_outer)
    boot_G_inner = np.sqrt(n)*mean_inner/np.sqrt(ssq_inner)

    # Work out weights
    outer_weights = AcHat_bdry_weights_concat[...,0]
    inner_weights = AcHat_bdry_weights_concat[...,1]


    # Work out interpolated values
    boot_G_bdry_interp_concat = inner_weights*boot_G_inner + outer_weights*boot_G_outer

    # Get bootstrap estimate of integral
    integral_value = np.sum(sigma_bdry_interp_concat*boot_G_bdry_interp_concat/muhat_grad_bdry_interp_concat)

    # Save integral value
    integral_values.append(integral_value)


In [ ]:
# Plot histogram of integral values
_ = plt.hist(integral_values, bins=100)

In [ ]:
print(clt_est)

In [ ]:
# Number of simulations
n_sim = 1000

# Initialize array for clt_est
clt_est_store = []

# Initialize array for integral values
sim_integral_values = []

# Empirical cdf
for i in np.arange(n_sim):

    print(i)

    # Output directory (feel free to change this to your desired output directory)
    out_dir = os.path.join(os.getcwd(),'data','example_2D')
    
    # Specify the noise 
    noise = Noise(var=1,fwhm=[6,6],n=100)   

    # We can generate some 2D data using the generate_data_2D function
    data_files, mu_file = generate_data_2D(signal, noise, out_dir=out_dir)

    # Let's record the number of observations; this will be useful later
    n = len(data_files)

    # Construct a design matrix with only an intercept (this means that the parameter estimate from the regression will be the mean of the data)
    X = np.ones((n,1))

    # Perform regression to get residuals, muhat, and sigma
    muhat_file, sigma_file, resid_files = regression(data_files, X, out_dir=out_dir)

    # Read in muhat and sigma
    muhat = read_image(muhat_file)
    sigma = read_image(sigma_file)

    # Compute G = (muhat-mu)/(sigma/sqrt(n))
    G = (muhat-mu)/(sigma/np.sqrt(n))

    # Get tau
    tau = 1/np.sqrt(n) 

    # Ac 
    Ac = mu > c

    # AcHat
    AcHat = muhat > c

    # Get size of estimated cluster
    est_clus_size = np.sum(AcHat)

    # Get size of true cluster
    true_clus_size = np.sum(Ac)

    # Get estimated clt value
    clt_est = (est_clus_size - true_clus_size)/tau

    # Record clt_est
    clt_est_store.append(clt_est)

    # Get boolean maps for the boundary of AcHat
    AcHat_bdry_map = get_bdry_maps(muhat, c)

    # Get coordinates for the boundary of AcHat
    AcHat_bdry_locs = get_bdry_locs(AcHat_bdry_map)

    # Obtain the muhat values along the boundary for AcHat
    muhat_bdry_vals_concat = get_bdry_values_concat(muhat, AcHat_bdry_locs)

    # Obtain the weights along the boundary for AcHat
    AcHat_bdry_weights_concat = get_bdry_weights_concat(muhat_bdry_vals_concat, c)

    # Get G values along the boundary of AcHat
    G_bdry_vals_concat = get_bdry_values_concat(G, AcHat_bdry_locs)

    # Interpolate G along the boundary of AcHat
    G_bdry_interp_concat = get_bdry_vals_interpolated_concat(G_bdry_vals_concat, AcHat_bdry_weights_concat)

    # Obtain the sigmahat values along the boundary for AcHat
    sigma_bdry_vals_concat = get_bdry_values_concat(sigma, AcHat_bdry_locs)

    # Interpolate sigma along the boundary of AcHat
    sigma_bdry_interp_concat = get_bdry_vals_interpolated_concat(sigma_bdry_vals_concat, AcHat_bdry_weights_concat)

    # Obtain the gradient of muhat from numpy
    muhat_grad = np.gradient(muhat)

    # Obtain the magnitude of the gradient from the partial derivatives
    muhat_grad = np.sqrt(muhat_grad[0]**2 + muhat_grad[1]**2)

    # Get muhat_grad values along the boundary of AcHat
    muhat_grad_bdry_vals_concat = get_bdry_values_concat(muhat_grad, AcHat_bdry_locs)

    # Interpolate muhat_grad along the boundary of AcHat
    muhat_grad_bdry_interp_concat = get_bdry_vals_interpolated_concat(muhat_grad_bdry_vals_concat, AcHat_bdry_weights_concat)

    # Integral value
    sim_integral_value = np.sum(sigma_bdry_interp_concat*G_bdry_interp_concat/muhat_grad_bdry_interp_concat)

    # Save integral value
    sim_integral_values.append(sim_integral_value)


In [ ]:
# Make a histogram of the clt_est values
_ = plt.hist(clt_est_store, bins=100)

In [ ]:
# Make a histogram of sim_integral_values
_ = plt.hist(sim_integral_values, bins=100)

In [ ]:
_ = plt.hist(integral_values, bins=100, density=True)

In [ ]:
plt.plot(np.sort(clt_est_store), np.linspace(0, 1, len(clt_est_store), endpoint=False))


In [ ]:
plt.plot(np.sort(integral_values), np.linspace(0, 1, len(integral_values), endpoint=False))

In [ ]:
sorted_clt_est_store = np.sort(clt_est_store)
sorted_integral_values = np.sort(integral_values)

print(len(integral_values))
print(len(clt_est_store))

for i in np.arange(len(clt_est_store)):

    print(sorted_integral_values[5*i]/sorted_clt_est_store[i])

In [ ]:
# Make a plot of muhat
plt.imshow(muhat)
plt.colorbar()

In [ ]:
# Obtain the gradient of muhat from numpy
muhat_grad = np.gradient(muhat)

# Obtain the magnitude of the gradient from the partial derivatives
muhat_grad = np.sqrt(muhat_grad[0]**2 + muhat_grad[1]**2)

# Plot the gradient
plt.imshow(muhat_grad)

### Function Versions

In [16]:
def bootstrap(resids,bdry_weights,sigma_bdry_interp_concat,muhat_grad_bdry_interp_concat,n,n_boot=5000):

    # Dimensions of bootstrap variables
    boot_dim = np.array([n, 1, 1]) 

    # Initialize numpy array of zeros to store integral values
    integral_values = np.zeros((1,n_boot))

    # Perform bootstrap
    for b in np.arange(n_boot):

        print('Boot: ' + str(b))

        # Obtain bootstrap variables
        boot_vars = 2*np.random.randint(0,2,boot_dim,dtype="int8")-1

        # ------------------------------------------------------
        # Bootstrap residuals
        # ------------------------------------------------------

        # Multiply by rademacher variables
        boot_resids = boot_vars*resids

        # ------------------------------------------------------
        # Get gi along dalpha FcHat
        # ------------------------------------------------------
        # Sum across subjects to get the bootstrapped a values along
        # the boundary of dalphaFcHat. (Note: For some reason this is 
        # much faster if performed seperately for each of the last rows. 
        # I am still looking into why this is)
        mean_outer = np.sum(boot_resids[...,0], axis=0)/n
        mean_inner = np.sum(boot_resids[...,1], axis=0)/n

        # Get sum of squares
        ssq_outer = np.sum((boot_resids[...,0]-mean_outer)**2, axis=0)/(n-1)
        ssq_inner = np.sum((boot_resids[...,1]-mean_inner)**2, axis=0)/(n-1)

        # Obtain bootstrap G
        boot_G_outer = np.sqrt(n)*mean_outer/np.sqrt(ssq_outer)
        boot_G_inner = np.sqrt(n)*mean_inner/np.sqrt(ssq_inner)

        # Work out weights
        outer_weights = bdry_weights[...,0]
        inner_weights = bdry_weights[...,1]


        # Work out interpolated values
        boot_G_bdry_interp_concat = inner_weights*boot_G_inner + outer_weights*boot_G_outer

        # Get bootstrap estimate of integral
        integral_value = np.sum(sigma_bdry_interp_concat*boot_G_bdry_interp_concat/muhat_grad_bdry_interp_concat)

        # Save integral value
        integral_values[0,b] = integral_value

    # Sort integral values
    integral_values[0,:] = np.sort(integral_values[0,:])

    # Return integral values
    return integral_values

In [17]:

# This function will run n_sim simulations and save the results to a csv file.
def run_simulation(n_sim, n, out_dir,n_boot=5000):

    # Initialize array of zeros of length n_sim for clt_est
    clt_est_store = np.zeros(n_sim)

    # Initialize array of zeros of length n_sim for integral values
    sim_integral_values = np.zeros(n_sim)

    # Initialize array of zeros of size (n_sim,n_boot) for boot integral values
    boot_integral_values = np.zeros((n_sim,n_boot))

    # Output directory (feel free to change this to your desired output directory)
    out_dir = os.path.join(os.getcwd(),'data','example_2D')

    # Empirical cdf
    for i in np.arange(n_sim):

        print('Sim: ' + str(i))
        
        # This command will define a circular smooth signal
        signal = CircleSignal(r=25, fwhm=[10,10], mag=3)

        # Generate signal
        signal.generate()

        # Get signal
        mu = signal.mu
        
        # Specify the noise 
        noise = Noise(var=1,fwhm=[6,6],n=n)   
 
        # Threshold c
        c = 2

        # We can generate some 2D data using the generate_data_2D function
        data_files, mu_file = generate_data_2D(signal, noise, out_dir=out_dir)

        # Construct a design matrix with only an intercept (this means that the parameter estimate from the regression will be the mean of the data)
        X = np.ones((n,1))

        # Perform regression to get residuals, muhat, and sigma
        muhat_file, sigma_file, resid_files = regression(data_files, X, out_dir=out_dir)

        # Read in muhat and sigma
        muhat = read_image(muhat_file)
        sigma = read_image(sigma_file)

        # Compute G = (muhat-mu)/(sigma/sqrt(n))
        G = (muhat-mu)/(sigma/np.sqrt(n))

        # Get tau
        tau = 1/np.sqrt(n) 

        # Ac 
        Ac = mu > c

        # AcHat
        AcHat = muhat > c

        # Get size of estimated cluster
        est_clus_size = np.sum(AcHat)

        # Get size of true cluster
        true_clus_size = np.sum(Ac)

        # Get estimated clt value
        clt_est = (est_clus_size - true_clus_size)/tau

        # Record clt_est
        clt_est_store[i] = clt_est

        # Get boolean maps for the boundary of AcHat
        AcHat_bdry_map = get_bdry_maps(muhat, c)

        # Get coordinates for the boundary of AcHat
        AcHat_bdry_locs = get_bdry_locs(AcHat_bdry_map)

        # Obtain the muhat values along the boundary for AcHat
        muhat_bdry_vals_concat = get_bdry_values_concat(muhat, AcHat_bdry_locs)

        # Obtain the weights along the boundary for AcHat
        AcHat_bdry_weights_concat = get_bdry_weights_concat(muhat_bdry_vals_concat, c)

        # Get G values along the boundary of AcHat
        G_bdry_vals_concat = get_bdry_values_concat(G, AcHat_bdry_locs)

        # Interpolate G along the boundary of AcHat
        G_bdry_interp_concat = get_bdry_vals_interpolated_concat(G_bdry_vals_concat, AcHat_bdry_weights_concat)

        # Obtain the sigmahat values along the boundary for AcHat
        sigma_bdry_vals_concat = get_bdry_values_concat(sigma, AcHat_bdry_locs)

        # Interpolate sigma along the boundary of AcHat
        sigma_bdry_interp_concat = get_bdry_vals_interpolated_concat(sigma_bdry_vals_concat, AcHat_bdry_weights_concat)

        # Obtain the gradient of muhat from numpy
        muhat_grad = np.gradient(muhat)

        # Obtain the magnitude of the gradient from the partial derivatives
        muhat_grad = np.sqrt(muhat_grad[0]**2 + muhat_grad[1]**2)

        # Get muhat_grad values along the boundary of AcHat
        muhat_grad_bdry_vals_concat = get_bdry_values_concat(muhat_grad, AcHat_bdry_locs)

        # Interpolate muhat_grad along the boundary of AcHat
        muhat_grad_bdry_interp_concat = get_bdry_vals_interpolated_concat(muhat_grad_bdry_vals_concat, AcHat_bdry_weights_concat)

        # Integral value
        sim_integral_value = np.sum(sigma_bdry_interp_concat*G_bdry_interp_concat/muhat_grad_bdry_interp_concat)

        # Save integral value
        sim_integral_values[i] = sim_integral_value

        # Loop through subjects
        for j in np.arange(n):

            # Obtain residuals
            resid = read_images(resid_files[j])[...,0]

            # Standardize residuals
            resid = (resid/sigma).reshape((1,) + resid.shape)

            # Residuals along boundary for current subject
            resid_bdry_concat_j = get_bdry_values_concat(resid, AcHat_bdry_locs)

            # Concatenate residuals
            if j == 0:
                resids = resid_bdry_concat_j
            else:
                resids = np.concatenate((resids, resid_bdry_concat_j), axis=0)

        # Run bootstrap
        boot_integral_values = bootstrap(resids, AcHat_bdry_weights_concat, sigma_bdry_interp_concat,muhat_grad_bdry_interp_concat, n, n_boot=n_boot)

        # Save integral values
        boot_integral_values[i,:] = boot_integral_values

    # Sort clt_est_store
    clt_est_store = np.sort(clt_est_store)

    # Save clt_est_store
    append_to_file(os.path.join(out_dir,'clt_est_store.csv'), clt_est_store)

    # Sort sim_integral_values
    sim_integral_values = np.sort(sim_integral_values)

    # Save sim_integral_values
    append_to_file(os.path.join(out_dir,'sim_integral_values.csv'), sim_integral_values)

    # Save boot_integral_values
    append_to_file(os.path.join(out_dir,'boot_integral_values.csv'), boot_integral_values)

    # Remove other files
    remove_files(mu_file, data_files, muhat_file, sigma_file, resid_files)
        

In [18]:

# Output directory (feel free to change this to your desired output directory)
out_dir = os.path.join(os.getcwd(),'data','example_2D')

# Run a simulation with 1 instance and 100 subjects
run_simulation(1, 100, out_dir, 5000)

Sim: 0
Boot: 0
Boot: 1
Boot: 2
Boot: 3
Boot: 4
Boot: 5
Boot: 6
Boot: 7
Boot: 8
Boot: 9
Boot: 10
Boot: 11
Boot: 12
Boot: 13
Boot: 14
Boot: 15
Boot: 16
Boot: 17
Boot: 18
Boot: 19
Boot: 20
Boot: 21
Boot: 22
Boot: 23
Boot: 24
Boot: 25
Boot: 26
Boot: 27
Boot: 28
Boot: 29
Boot: 30
Boot: 31
Boot: 32
Boot: 33
Boot: 34
Boot: 35
Boot: 36
Boot: 37
Boot: 38
Boot: 39
Boot: 40
Boot: 41
Boot: 42
Boot: 43
Boot: 44
Boot: 45
Boot: 46
Boot: 47
Boot: 48
Boot: 49
Boot: 50
Boot: 51
Boot: 52
Boot: 53
Boot: 54
Boot: 55
Boot: 56
Boot: 57
Boot: 58
Boot: 59
Boot: 60
Boot: 61
Boot: 62
Boot: 63
Boot: 64
Boot: 65
Boot: 66
Boot: 67
Boot: 68
Boot: 69
Boot: 70
Boot: 71
Boot: 72
Boot: 73
Boot: 74
Boot: 75
Boot: 76
Boot: 77
Boot: 78
Boot: 79
Boot: 80
Boot: 81
Boot: 82
Boot: 83
Boot: 84
Boot: 85
Boot: 86
Boot: 87
Boot: 88
Boot: 89
Boot: 90
Boot: 91
Boot: 92
Boot: 93
Boot: 94
Boot: 95
Boot: 96
Boot: 97
Boot: 98
Boot: 99
Boot: 100
Boot: 101
Boot: 102
Boot: 103
Boot: 104
Boot: 105
Boot: 106
Boot: 107
Boot: 108
Boot: 109
Boo

KeyboardInterrupt: 